In [ ]:
# Importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import catboost as cb
warnings.filterwarnings('ignore')

import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from skopt import BayesSearchCV
import pprint
from time import time
from sklearn.metrics import accuracy_score
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestRegressor
from collections import defaultdict
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


In [ ]:
# loading data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# Combine dataset here
test['amount_spent_per_room_night_scaled'] = np.nan

comb = train.append(test)

In [ ]:
comb['odd_adults'] = np.where(comb['numberofadults'] % 2 !=0 , 1 ,0)

In [ ]:
# Handle missing values here
comb['state_code_residence'] = np.where(comb['state_code_residence'].isnull(),8.0,comb['state_code_residence'])  # Filling by mode
comb['season_holidayed_code'] = np.where(comb['season_holidayed_code'].isnull(),2.0,comb['season_holidayed_code'])  # Filling by mode


In [ ]:
# Data Cleaning and feature engineering

comb['booking_date'] = pd.to_datetime(comb['booking_date'], format='%d/%m/%y')
comb['checkin_date'] = pd.to_datetime(comb['checkin_date'], format='%d/%m/%y')
comb['checkout_date'] = pd.to_datetime(comb['checkout_date'], format='%d/%m/%y')

comb['duration_stayed'] = comb['checkout_date'] - comb['checkin_date']
comb['duration_stayed']=comb['duration_stayed'].dt.days
comb['duration_stayed'] = np.where(comb['duration_stayed'] >7,7,comb['duration_stayed'])
comb['weekday'] = np.where(comb['checkin_date'].dt.weekday > 4,1,0)

In [ ]:
comb['checkin_month'] = comb['checkin_date'].dt.month
comb['booking_checkin'] = comb['checkin_date'] - comb['booking_date']
comb['booking_checkin'] = comb['booking_checkin'].dt.days
comb['booking_checkin'] = np.where(comb['booking_checkin'] < 0,comb['booking_checkin'].median(),comb['booking_checkin'])
comb['booking_checkin'] = np.where(comb['booking_checkin'] > 3,3,comb['booking_checkin'])

In [ ]:
comb['totalpeople'] = comb['numberofadults'] + comb['numberofchildren'] 
comb['totalpeople'] = np.where(comb['totalpeople']>6,6,comb['totalpeople'])
comb['totalpeople'] = comb['totalpeople'].replace({0:4})

In [ ]:
comb['roomnights'] = np.where(comb['roomnights'] > 7 ,7,comb['roomnights'])
comb['booked_stayed'] = comb['roomnights'] - comb['duration_stayed']

In [ ]:
comb['reservationstatusid_code'] = np.where(comb['reservationstatusid_code'] == 'D','C',comb['reservationstatusid_code'])
comb['persontravellingid'] = comb['persontravellingid'].replace({4995:4753})
comb['room_type_booked_code'] = comb['room_type_booked_code'].replace({5:6})
comb['season_holidayed_code'] = comb['season_holidayed_code'].astype('int')
comb['total_pax'] = np.where(comb['total_pax'] > 7 ,7,comb['total_pax'])

In [ ]:
comb['state_code_residence']  = comb['state_code_residence'].astype('int')
# comb['state_code_residence'] = comb['state_code_residence'].replace({22 :28,24:28,30:28,29:28,33:28,34:28,32:28,37:28,31:28,35:28,
#                                                                     36:28,38:28})

In [ ]:
comb = comb.drop(['booking_date','checkin_date','checkout_date','numberofadults','numberofchildren','memberid'],axis = 1)

In [ ]:
comb = pd.get_dummies(comb,columns=['channel_code','main_product_code','persontravellingid','resort_region_code',
                                   'resort_type_code','room_type_booked_code','season_holidayed_code','state_code_residence',
                                   'state_code_resort','member_age_buckets','booking_type_code','cluster_code',
                                   'reservationstatusid_code','resort_id'])


In [ ]:
trainset = comb[comb['amount_spent_per_room_night_scaled'].notnull()]
testset = comb[comb['amount_spent_per_room_night_scaled'].isnull()]

X = trainset.drop(['reservation_id','amount_spent_per_room_night_scaled'],axis = 1)
y = pd.DataFrame(trainset['amount_spent_per_room_night_scaled'])

In [ ]:
# Keeping eval set aside
X_trainset,X_eval,y_trainset,y_eval = train_test_split(X,y,test_size = 0.15,random_state = 25)

In [ ]:
# Breaking into train and test
X_train,X_test,y_train,y_test = train_test_split(X_trainset,y_trainset,test_size = 0.20,random_state = 11)

In [ ]:
# XGBoost Tuning
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ['rmse']

In [ ]:
model = xgb.XGBRegressor(
                      learning_rate=0.03,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='reg:linear', 
                      n_estimators=100, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)

model.fit(X_train,y_train,eval_set = eval_set,eval_metric = eval_metric)

In [ ]:
reg = xgb.XGBRegressor()
reg.fit(X_train,y_train)

In [ ]:
reg.feature_importances_

In [ ]:

#Perform cross-validation:
# cv_score = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
# cv_score = np.sqrt(np.abs(cv_score))

#Print model report:
print ("\nModel Report")
print ("RMSE : %.4g" % np.sqrt(mean_squared_error(y_train, reg.predict(X_train))))
# print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))

rms = sqrt(mean_squared_error(y_test, reg.predict(X_test)))
print('Validation rms {}'.format(rms))

In [ ]:
sub = pd.DataFrame()
sub['reservation_id'] = testset['reservation_id']
pred = reg.predict(testset.drop(['reservation_id','amount_spent_per_room_night_scaled'],axis = 1))
sub['amount_spent_per_room_night_scaled'] = pred
sub.to_csv('win1.csv',index =False)